In [6]:
import os
import torch
import torch.nn.functional as F

def get_features(pt_dir):
    pt_files = sorted([os.path.join(pt_dir, f) for f in os.listdir(pt_dir)])
    features = []
    for file in pt_files[:2000]:
        data = torch.load(file)
        image_features = data['image_features'].cpu()
        features.append(image_features)

    features = torch.cat(features)
    return features

root_dir = '/data/archive/sd-v1-4'
scales = [1.5, 3.5, 5.5, 7.5, 9.5]
steps_list = [5, 6, 8, 10, 12, 15, 20]
#model_names = ['dpm_solver++', 'uni_pc_bh2', 'dpm_solver_v3', 'rbf_order2', 'rbf_order3']
model_names = ['dcsolver_order3']

for scale in scales:  
    print(scale)
    path = os.path.join(root_dir, f"dpm_solver++_steps200_scale{scale}_clip")
    ref_features = get_features(path)

    for model_name in model_names:
        cosims = []
        for steps in steps_list:
            path = os.path.join(root_dir, f"{model_name}_steps{steps}_scale{scale}_clip")
            if os.path.exists(path):
                comp_features = get_features(path)
                cosim = torch.mean(F.cosine_similarity(ref_features, comp_features)).item()
                cosims.append(f"{cosim:0.4f}")
        if len(cosims) > 0:
            print(model_name, 'SPLIT("', ",".join(cosims), '",",")')
    print()


1.5
dcsolver_order3 SPLIT(" 0.8979,0.9214,0.9497,0.9678,0.9800,0.9878,0.9941 ",",")

3.5
dcsolver_order3 SPLIT(" 0.8916,0.9087,0.9277,0.9463,0.9561,0.9673,0.9790 ",",")

5.5
dcsolver_order3 SPLIT(" 0.8877,0.8887,0.9175,0.9321,0.9419,0.9487,0.9644 ",",")

7.5
dcsolver_order3 SPLIT(" 0.8721,0.8809,0.9004,0.9204,0.9238,0.9424,0.9497 ",",")

9.5
dcsolver_order3 SPLIT(" 0.8218,0.8540,0.8945,0.9062,0.9155,0.9268,0.9385 ",",")



In [7]:
!ls /data/archive/sd-v1-4

dpm_solver++_steps10_scale1.5	     rbf_order2_steps6_scale5.5
dpm_solver++_steps10_scale1.5_clip   rbf_order2_steps6_scale5.5_clip
dpm_solver++_steps10_scale1.5_fid    rbf_order2_steps6_scale9.5
dpm_solver++_steps10_scale3.5	     rbf_order2_steps6_scale9.5_clip
dpm_solver++_steps10_scale3.5_clip   rbf_order2_steps8_scale1.5
dpm_solver++_steps10_scale5.5	     rbf_order2_steps8_scale1.5_clip
dpm_solver++_steps10_scale5.5_clip   rbf_order2_steps8_scale3.5
dpm_solver++_steps10_scale9.5	     rbf_order2_steps8_scale3.5_clip
dpm_solver++_steps10_scale9.5_clip   rbf_order2_steps8_scale5.5
dpm_solver++_steps12_scale1.5	     rbf_order2_steps8_scale5.5_clip
dpm_solver++_steps12_scale1.5_clip   rbf_order2_steps8_scale9.5
dpm_solver++_steps12_scale3.5	     rbf_order2_steps8_scale9.5_clip
dpm_solver++_steps12_scale3.5_clip   rbf_order3_steps10_scale1.5
dpm_solver++_steps12_scale5.5	     rbf_order3_steps10_scale1.5_clip
dpm_solver++_steps12_scale5.5_clip   rbf_order3_steps10_scale3.5
dpm_solver++_ste

In [ ]:
!ls /dat